In [ ]:
!pip install pyannote.audio

In [ ]:
!pip install torchvision

In [ ]:
! pip install python-dotenv
! pip install pydub
! pip install pandas
! pip install psycopg2
! pip install google-auth google-auth-httplib2 google-api-python-client google-auth-oauthlib

In [ ]:
import sys
sys.path.append('../util')

from util import read_spreadsheet
from util import collect_segments
from util import split_audio_files

In [ ]:
import requests
import os
import subprocess

def download_url_file(url, local_filename):
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open(local_filename.strip(), 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        print(f"Downloaded {local_filename.strip()}")
    else:
        print(f"Failed to download {local_filename.strip()} from {url}. Status code: {response.status_code}")


In [ ]:
from pydub import AudioSegment
import pandas as pd

# Function to get audio duration from an audio file
def get_audio_duration(audio_filename):
    try:
        audio = AudioSegment.from_file(audio_filename)
        duration_in_sec = len(audio) // 1000  # Duration in seconds
        duration = pd.to_datetime(duration_in_sec, unit='s').strftime('%H:%M:%S')  # Duration in H:M:S format
        return duration
    except Exception as e:
        print(f"Error fetching duration for {audio_filename}: {e}")
        return None


In [ ]:
# Ensure directories exist
os.makedirs('audio_from_url', exist_ok=True)

# Example usage:
from_id = 163 
to_id = 201

durations = []

df = read_spreadsheet(sheet_id="1SjlnWsr6YVhBIW6EaS0ITmboDmEuj2VnxGRNL50OPx4")

for index, row in df.iterrows():
    if not isinstance(row['ID'], str) or not isinstance(row['YouTube link'], str):
        break
    id = row['ID']
    url_path = row['YouTube link']
    audio_filename = f"audio_from_url/{id}.mp3".strip()
    sr_no = row.iloc[0]
    if sr_no >= from_id and sr_no <= to_id:
        if os.path.exists(audio_filename):
            print(f"Audio file {audio_filename} already exists. Skipping extraction.")
            continue
        if not os.path.exists(audio_filename):
            download_url_file(url_path, audio_filename)
        duration = get_audio_duration(audio_filename)
        print(f"Audio duration for {audio_filename}: {duration}")
        durations.append((id, duration))



In [ ]:
# Convert list to DataFrame and save to CSV
df_durations = pd.DataFrame(durations, columns=['File Name', 'Duration'])
df_durations.to_csv('nw_gh_audio_durations.csv', index=False)

print("Durations have been saved to audio_durations.csv")